In [1]:
import json, collections, time
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="location_converter")

# Read the JSON file
with open('raw.json', 'r') as f:
    data = json.load(f)
    

In [3]:
with open('../locations.json', 'r') as f:
    location_cache = json.load(f)

PREDEFINED_MAPPING = {"Austin, Texas Metropolitan Area": "Austin, Texas, USA",}

def get_coord(location):
    if location_cache.get(location) is not None:
        return location_cache[location]

    print(f"API Call! for {location}")
    geolocator = Nominatim(user_agent="location_converter")
    time.sleep(1)

    loc = geolocator.geocode(location)
    assert loc, f"Location not found, {loc.raw}"
    
    location_cache[location] = [[loc.latitude, loc.longitude, loc.address]]
    return location_cache[location]

location_coordinates = {}

for i in data:
    location = i['job_posting_location']
    #####################
    if "remote" in location.lower():
        location_coordinates[location] = [["remote", "remote", "remote"]]
        continue
    #####################
    split_by = " and "
    if split_by in location:
        location_coordinates[location] = []
        for loc in location.split(split_by):
            if "more" in loc:
                continue
            coord = get_coord(loc)
            location_coordinates[location].append(coord)
        continue
    split_by = " or "
    if split_by in location:
        location_coordinates[location] = []
        for loc in location.split(split_by):
            if "more" in loc:
                continue
            coord = get_coord(loc)
            location_coordinates[location].append(coord)
        continue
    #####################
    split_by = " | "
    if split_by in location:
        location_coordinates[location] = []
        for loc in location.split(split_by):
            coord = get_coord(loc)
            location_coordinates[location].append(coord)
        continue
    #####################
    split_by = ";"
    if split_by in location:
        locs = []
        for loc in location.split(split_by):
            coord = get_coord(loc)
            locs.append(coord)
        location_coordinates[location] = locs
        continue
    split_by = " • "
    if split_by in location:
        locs = []
        for loc in location.split(split_by):
            coord = get_coord(loc)
            locs.append(coord)
        location_coordinates[location] = locs
        continue
    # #####################
    # if "locations" in location.lower():
    #     # print(location)
    #     continue
    
    removals = ["Location pin icon ", "locations", "office", "Category"]
    clean_location = location
    for i in removals:
        clean_location = clean_location.replace(i, "")
        
    if clean_location in PREDEFINED_MAPPING:
        location_coordinates[location] = [get_coord(PREDEFINED_MAPPING[clean_location])]
        continue
    try:
        location_coordinates[location] = [get_coord(clean_location)]
    except Exception as e:
        print(f"Error: {location}")
with open('../locations.json', 'w') as f:
    location_coordinates.update(location_cache)
    json.dump(location_coordinates, f, indent=2)

API Call! for Austin, Texas, USA
API Call! for San Francisco Office
Error: San Francisco Office
API Call! for  Schnucks Store Support Center (Corporate Office)
Error: locations Schnucks Store Support Center (Corporate Office)
API Call! for USA - New York - Malta, USA
Error: USA - New York - Malta, USA
API Call! for OT494: 5FS - Farmington, CT 5 Farm Springs, Farmington, CT, 06032 USA
Error: OT494: 5FS - Farmington, CT 5 Farm Springs, Farmington, CT, 06032 USA
API Call! for  Dublin, OH / Hybrid
Error: locations Dublin, OH / Hybrid
API Call! for Seattle, Washington United States, USA
Error: Seattle, Washington United States, USA
API Call! for  Pleasant Prairie, WI Waukegan, IL Glenview, IL Milwaukee, WI Kenosha, WI
Error: locations Pleasant Prairie, WI Waukegan, IL Glenview, IL Milwaukee, WI Kenosha, WI
API Call! for USA - Rhode Island - East Providence, USA
Error: USA - Rhode Island - East Providence, USA
API Call! for San Francisco, CA New York, NY
Error: San Francisco, CA New York, NY

AttributeError: 'NoneType' object has no attribute 'raw'

In [4]:

with open('../locations.json', 'w') as f:
    location_coordinates.update(location_cache)
    json.dump(location_coordinates, f, indent=2)

In [69]:
# Load the JSON file
with open('../locations.json', 'r') as f:
	location_data = json.load(f)

# Normalize the structure to ensure all values are lists of lists
for key, value in location_data.items():
	# If the value is a list but not a list of lists
	if isinstance(value, list) and value and not isinstance(value[0], list):
		location_data[key] = [value]

# Save the normalized data back to the file
with open('../locations.json', 'w') as f:
	json.dump(location_data, f, indent=2)
